In [2]:
import pandas as pd

In [3]:
df  = pd.read_csv('/content/datasets_4123_6408_framingham.csv')
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [4]:
df.shape


(4240, 16)

In [6]:
df.drop('education' , axis  =1 , inplace=True)

In [7]:
df.isnull().sum()

,0
male,0
age,0
currentSmoker,0
cigsPerDay,29
BPMeds,53
prevalentStroke,0
prevalentHyp,0
diabetes,0
totChol,50
sysBP,0




---

# Fill Missing Values


In [8]:
# Define the binary columns
bin_cols = ["male", "currentSmoker", "prevalentStroke", "prevalentHyp", "diabetes"]

# Fill missing values for binary features with the most frequent value (mode)
for col in bin_cols:
    mode_val = df[col].mode()[0]
    df[col].fillna(mode_val, inplace=True)

# Check if there are any remaining missing values
missing_values = df.isnull().sum()

<ipython-input-8-564253c0296b>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mode_val, inplace=True)
<ipython-input-8-564253c0296b>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method

In [10]:
# Import necessary libraries
import numpy as np

# Fill missing values for numeric features
numeric_cols = ["cigsPerDay", "BPMeds", "totChol", "BMI", "heartRate", "glucose"]
for col in numeric_cols:
    median_val = df[col].median()
    df[col].fillna(median_val, inplace=True)


# Check if there are any remaining missing values
missing_values = df.isnull().sum()

<ipython-input-10-e4cfa7810365>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(median_val, inplace=True)


In [12]:
df['TenYearCHD'].value_counts()

,count
TenYearCHD,
0,3596
1,644


In [13]:
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df[df['TenYearCHD'] == 0]
df_minority = df[df['TenYearCHD'] == 1]

# Upsample minority class
df_minority_upsampled = resample(df_minority,
                                 replace=True,     # Sample with replacement
                                 n_samples=len(df_majority),    # To match majority class
                                 random_state=42) # Reproducible results

# Combine majority class with upsampled minority class
df_balanced = pd.concat([df_majority, df_minority_upsampled])

In [14]:
df_balanced['TenYearCHD'].value_counts()


,count
TenYearCHD,
0,3596
1,3596


# Train Test Split and Feature Scalling


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separate features (X) and target variable (y)
X = df_balanced.drop(columns=['TenYearCHD'])
y = df_balanced['TenYearCHD']

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
X_train

,male,age,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose
1905,0,64,0,0.0,0.0,0,1,0,229.0,145.0,85.0,29.67,70.0,74.0
2075,0,37,1,20.0,0.0,0,0,0,166.0,112.0,73.5,21.64,75.0,93.0
1128,0,63,1,10.0,0.0,0,1,0,236.0,189.0,103.0,27.91,60.0,74.0
1782,0,65,0,0.0,0.0,0,1,0,245.0,171.0,89.0,23.07,82.0,93.0
241,1,65,1,15.0,0.0,0,1,0,219.0,148.0,90.0,29.35,77.0,97.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,1,63,1,20.0,0.0,0,1,0,269.0,180.0,101.0,24.42,72.0,84.0
485,1,54,1,40.0,0.0,0,0,0,230.0,145.0,90.0,25.72,75.0,85.0
4232,1,68,0,0.0,0.0,0,1,0,176.0,168.0,97.0,23.14,60.0,79.0
952,1,66,1,30.0,0.0,0,0,1,234.0,114.5,62.5,28.62,75.0,216.0


In [17]:
X_test

,male,age,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose
3227,0,43,1,15.0,0.0,0,0,0,300.0,120.0,78.0,28.18,75.0,106.0
4221,1,50,0,0.0,0.0,0,0,0,260.0,119.0,74.0,21.85,80.0,72.0
1771,1,57,1,20.0,0.0,0,0,0,193.0,104.0,64.0,26.00,62.0,87.0
3805,1,51,1,40.0,0.0,0,1,0,227.0,162.5,104.0,34.97,90.0,65.0
1902,0,52,1,20.0,1.0,0,1,0,161.0,180.0,114.0,32.52,105.0,104.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,0,38,1,3.0,1.0,0,1,0,234.0,125.0,80.0,22.79,98.0,78.0
581,1,62,1,3.0,0.0,0,0,0,234.0,117.0,80.0,26.97,78.0,67.0
3979,0,44,0,0.0,0.0,0,0,0,180.0,120.0,80.0,21.67,60.0,78.0
2903,0,57,1,20.0,0.0,0,0,0,174.0,120.0,62.0,25.13,95.0,77.0


In [18]:
# Initialize StandardScaler
scaler = StandardScaler()

# Fit scaler to training data and transform both training and testing data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [19]:
X_train_scaled

array([[-0.94172615,  1.4582083 , -1.02624378, ...,  0.840088  ,
        -0.50731448, -0.34072887],
       [-0.94172615, -1.66694628,  0.97442734, ..., -1.050363  ,
        -0.0898974 ,  0.25133934],
       [-0.94172615,  1.34246184,  0.97442734, ...,  0.42574258,
        -1.34214864, -0.34072887],
       ...,
       [ 1.06187982,  1.92119417, -1.02624378, ..., -0.6972277 ,
        -1.34214864, -0.18492145],
       [ 1.06187982,  1.68970124,  0.97442734, ...,  0.59289329,
        -0.0898974 ,  4.08420194],
       [ 1.06187982, -0.74097455,  0.97442734, ...,  1.0660946 ,
         0.32751968,  1.84057505]])

# Training 10 Models With Different Metrics


In [20]:
from sklearn.ensemble import  RandomForestClassifier

rf_model = RandomForestClassifier()
rf_model.fit(X_train_scaled , y_train)
y_pred = rf_model.predict(X_test_scaled)

In [21]:
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [22]:
from sklearn.metrics import  accuracy_score , confusion_matrix , classification_report


In [23]:
accuracy_score(y_test , y_pred)

0.9728978457261988

In [24]:
confusion_matrix(y_test , y_pred)

array([[701,  34],
       [  5, 699]])

In [25]:
print(classification_report(y_test  , y_pred))

              precision    recall  f1-score   support

           0       0.99      0.95      0.97       735
           1       0.95      0.99      0.97       704

    accuracy                           0.97      1439
   macro avg       0.97      0.97      0.97      1439
weighted avg       0.97      0.97      0.97      1439



In [26]:
from sklearn.ensemble import RandomForestClassifier , AdaBoostClassifier , GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

In [27]:
# Define a list of classifiers
classifiers = [
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    LogisticRegression(),
    SVC(),
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    GaussianNB(),
    XGBClassifier()
]
# Create a dictionary to store the results
results = {}


# Train and evaluate each classifier
for clf in classifiers:
    clf_name = clf.__class__.__name__
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{clf_name} Accuracy: {accuracy}")

    # Classification report
    print(f"Classification Report for {clf_name}:")
    print(classification_report(y_test, y_pred))

    # Confusion matrix
    print(f"Confusion Matrix for {clf_name}:")
    print(confusion_matrix(y_test, y_pred))
    print("="*50)

RandomForestClassifier Accuracy: 0.9763724808895066
Classification Report for RandomForestClassifier:
              precision    recall  f1-score   support

           0       0.99      0.96      0.98       735
           1       0.96      0.99      0.98       704

    accuracy                           0.98      1439
   macro avg       0.98      0.98      0.98      1439
weighted avg       0.98      0.98      0.98      1439

Confusion Matrix for RandomForestClassifier:
[[706  29]
 [  5 699]]
AdaBoostClassifier Accuracy: 0.6518415566365532
Classification Report for AdaBoostClassifier:
              precision    recall  f1-score   support

           0       0.68      0.61      0.64       735
           1       0.63      0.70      0.66       704

    accuracy                           0.65      1439
   macro avg       0.65      0.65      0.65      1439
weighted avg       0.65      0.65      0.65      1439

Confusion Matrix for AdaBoostClassifier:
[[447 288]
 [213 491]]
GradientBoostingCl

# Show Each Model Results


In [31]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Define a list of classifiers
classifiers = [
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    LogisticRegression(),
    SVC(),
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    GaussianNB(),
    XGBClassifier()
]

# Create an empty list to collect results
results = []

# Train and evaluate each classifier
for clf in classifiers:
    clf_name = clf.__class__.__name__
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    f1_score = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']

    # Append results as a dictionary
    results.append({
        'Model': clf_name,
        'Accuracy': accuracy,
        'F1-Score': f1_score,
        'Precision': precision,
        'Recall': recall
    })

# Convert the list of results to a DataFrame
results_df = pd.DataFrame(results)

# Display the results
results_df


,Model,Accuracy,F1-Score,Precision,Recall
0,RandomForestClassifier,0.972203,0.972203,0.973049,0.972203
1,AdaBoostClassifier,0.651842,0.651286,0.654290,0.651842
2,GradientBoostingClassifier,0.728978,0.728702,0.731320,0.728978
3,LogisticRegression,0.658791,0.658830,0.659053,0.658791
4,SVC,0.683113,0.683126,0.683656,0.683113
5,KNeighborsClassifier,0.787352,0.783833,0.812481,0.787352
6,DecisionTreeClassifier,0.917999,0.917638,0.928593,0.917999
7,GaussianNB,0.583044,0.530092,0.635597,0.583044
8,XGBClassifier,0.906185,0.905977,0.912148,0.906185


In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Instantiate the RandomForestClassifier
rf_classifier = RandomForestClassifier()

# Train the RandomForestClassifier
rf_classifier.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred_rf = rf_classifier.predict(X_test_scaled)

# Calculate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Random Forest Classifier Accuracy:", accuracy_rf)

# Classification report
print("Classification Report for Random Forest Classifier:")
print(classification_report(y_test, y_pred_rf))

# Confusion matrix
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test, y_pred_rf))


Random Forest Classifier Accuracy: 0.9735927727588604
Classification Report for Random Forest Classifier:
              precision    recall  f1-score   support

           0       0.99      0.96      0.97       735
           1       0.95      0.99      0.97       704

    accuracy                           0.97      1439
   macro avg       0.97      0.97      0.97      1439
weighted avg       0.97      0.97      0.97      1439

Confusion Matrix for Random Forest Classifier:
[[702  33]
 [  5 699]]


# Single prediction


In [33]:
print("predcted class ",rf_classifier.predict(X_test_scaled[10].reshape(1,-1))[0])
print("actual class ", y_test.iloc[10])


predcted class  0
actual class  0


In [34]:
print("predcted class ",rf_classifier.predict(X_test_scaled[200].reshape(1,-1))[0])
print("actual class ", y_test.iloc[200])

predcted class  1
actual class  1


In [35]:
print("predcted class ",rf_classifier.predict(X_test_scaled[280].reshape(1,-1))[0])
print("actual class ", y_test.iloc[280])

predcted class  0
actual class  0


In [36]:
import pickle
pickle.dump(rf_classifier,open("rf_classifier.pkl",'wb'))
pickle.dump(scaler,open("scaler.pkl",'wb'))

In [37]:
import pickle

# Load the RandomForestClassifier model
with open("rf_classifier.pkl", "rb") as file:
    rf_classifier = pickle.load(file)

# Load the scaler
with open("scaler.pkl", "rb") as file:
    scaler = pickle.load(file)


In [38]:
import numpy as np

def predict(model, scaler, male, age, currentSmoker, cigsPerDay, BPMeds, prevalentStroke, prevalentHyp, diabetes, totChol, sysBP, diaBP, BMI, heartRate, glucose):
    # Encode categorical variables
    male_encoded = 1 if male.lower() == "male" else 0
    currentSmoker_encoded = 1 if currentSmoker.lower() == "yes" else 0
    BPMeds_encoded = 1 if BPMeds.lower() == "yes" else 0
    prevalentStroke_encoded = 1 if prevalentStroke.lower() == "yes" else 0
    prevalentHyp_encoded = 1 if prevalentHyp.lower() == "yes" else 0
    diabetes_encoded = 1 if diabetes.lower() == "yes" else 0

    # Prepare features array
    features = np.array([[male_encoded, age, currentSmoker_encoded, cigsPerDay, BPMeds_encoded, prevalentStroke_encoded, prevalentHyp_encoded, diabetes_encoded, totChol, sysBP, diaBP, BMI, heartRate, glucose]])

    # scalling
    scaled_features = scaler.transform(features)

    # predict by model
    result = model.predict(scaled_features)

    return result[0]

In [40]:
# test 1:
male = "female"
age = 56.00
currentSmoker = "yes"
cigsPerDay = 3.00
BPMeds = "no"
prevalentStroke = "no"
prevalentHyp = "yes"
diabetes = 'no'
totChol = 285.00
sysBP = 145.00
diaBP = 100.00
BMI = 30.14
heartRate = 80.00
glucose = 86.00


result = predict(rf_classifier, scaler, male, age, currentSmoker, cigsPerDay, BPMeds, prevalentStroke, prevalentHyp, diabetes, totChol, sysBP, diaBP, BMI, heartRate, glucose)


if result == 1:
    print("The Patient has Heart Diseases")
else:
    print("The Patiennt has No Heart Diseases")

The Patiennt has No Heart Diseases


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [41]:
import sklearn
sklearn.__version__


'1.6.1'